<h4> Ensemble Method </h4>
Each review is mapped to a feature vector by averaging the word embeddings of all words in the review. These features are then fed into a random forest classifier.

In [1]:
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve
from sklearn.pipeline import Pipeline

In [2]:
# data = os.path(r"Module 4/Supervised NN Project/train_model")
%store -r
reviews = pd.read_csv('_review_data.csv')
print(reviews.columns)
model.wv.most_similar("noir")

Index(['Unnamed: 0', 'description', 'two_outcomes'], dtype='object')


[('meunier', 0.8507343530654907),
 ('noirs', 0.7709870934486389),
 ('gris', 0.7536669969558716),
 ('bianco', 0.7067205309867859),
 ('grigio', 0.6630685329437256),
 ('nero', 0.5673935413360596),
 ('silkiness', 0.525829553604126),
 ('oregon', 0.4896107017993927),
 ('immature', 0.4749774932861328),
 ('california', 0.4512163996696472)]

In [3]:
w2v = model.wv
w2v.most_similar("good")

[('nice', 0.831934928894043),
 ('decent', 0.7923933863639832),
 ('proper', 0.7887036204338074),
 ('great', 0.7273436784744263),
 ('fine', 0.7250930666923523),
 ('excellent', 0.7233678102493286),
 ('adequate', 0.6736984848976135),
 ('solid', 0.660475492477417),
 ('fair', 0.5983995199203491),
 ('enough', 0.5882049798965454)]

In [4]:
print(w2v['nice'].shape)
num_features = 50

(50,)


In [5]:
# index2word_set = set(w2v.index2word)
# len(index2word_set)

In [6]:
def make_feature_vec(words, model, num_features):
    
#     Average the word vectors for a set of words
    
    feature_vec = np.zeros((num_features,),dtype="float32")
    nwords = 0.
    index2word_set = set(w2v.index2word)  # words known to the model

    for word in words:
        if word in index2word_set: 
            nwords = nwords + 1.
            feature_vec = np.add(feature_vec,model[word])
    
    feature_vec = np.divide(feature_vec, nwords)
    return feature_vec


def get_avg_feature_vecs(reviews, model, num_features):
    
#     Calculate average feature vectors for all wine reviews
    
    counter = 0
    desc_feature_vecs = np.zeros((len(reviews),num_features), dtype='float32')
    
    for desc in reviews:
        desc_feature_vecs[counter] = make_feature_vec(desc, model, num_features)
        counter = counter + 1
    return desc_feature_vecs


Word2Vec of description

In [7]:
print((reviews.shape[0] * 0.70))

90934.9


In [8]:
train_reviews = reviews[:90935]
test_reviews = reviews[90935:]
print(reviews.shape[0], len(train_reviews) + len(test_reviews))

129907 129907


In [9]:

# calculate average feature vectors for training and test sets

trainDataVecs = get_avg_feature_vecs(train_reviews['description'], model, num_features)

testDataVecs = get_avg_feature_vecs(test_reviews['description'], model, num_features)



C:\Users\Mark\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:12: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if sys.path[0] == '':


In [10]:
trainDataVecs[0]

array([ 0.08443289, -0.01748779, -0.18085916, -0.14912176,  0.00550791,
       -0.05918663, -0.05578178, -0.16033605,  0.07697774, -0.0874128 ,
       -0.06674305, -0.17707986, -0.06660268,  0.20295903, -0.12143666,
       -0.19695324,  0.09940048, -0.14379378, -0.17519991, -0.07587919,
        0.01958998,  0.06260195, -0.13870066, -0.11189912,  0.02611933,
        0.05223307,  0.13144659, -0.05290572,  0.08743954, -0.11106408,
       -0.14252013,  0.27590382,  0.05521486, -0.16753085,  0.17436348,
        0.01557619,  0.0399346 , -0.03053275,  0.1015004 , -0.24072695,
        0.10247923, -0.08319446,  0.01942956, -0.17561089, -0.10253713,
        0.15378103, -0.0026733 , -0.1250518 , -0.16158238,  0.08599287],
      dtype=float32)

In [11]:
%%time
# Fit a random forest to the training data, using 100 trees
forest = RandomForestClassifier(n_estimators = 100)

print("Fitting a random forest to labeled training data...")
forest = forest.fit(trainDataVecs, train_reviews['two_outcomes'])

Fitting a random forest to labeled training data...
Wall time: 1min 54s


In [12]:
# remove instances in test set that could not be represented as feature vectors
nan_indices = list({x for x,y in np.argwhere(np.isnan(testDataVecs))})
if len(nan_indices) > 0:
    print('Removing {:d} instances from test set.'.format(len(nan_indices)))
    testDataVecs = np.delete(testDataVecs, nan_indices, axis=0)
    test_reviews.drop(test_reviews.iloc[nan_indices, :].index, axis=0, inplace=True)
    assert testDataVecs.shape[0] == len(test_reviews)

In [13]:
print("Predicting labels for test data..")
result = forest.predict(testDataVecs)
print(classification_report(test_reviews['two_outcomes'], result))

Predicting labels for test data..


NameError: name 'forest' is not defined

In [ ]:
probs = forest.predict_proba(testDataVecs)[:, 1]

fpr, tpr, _ = roc_curve(test_reviews['two_outcomes'], probs)
auc = roc_auc_score(test_reviews['two_outcomes'], probs)

plt.figure(1)
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr, tpr, label='AUC {:.3f}'.format(auc))
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve')
plt.legend(loc='best')
plt.show()
plt.savefig('AUC ROC Wine Reviews')

In [ ]:
%store trainDataVecs
%store testDataVecs

In [ ]:
%store train_reviews
%store test_reviews

In [ ]:
trainDataVecs.shape

In [ ]:
train_reviews.shape

In [ ]:


def plot_confusion_matrix(y_true, y_pred, classes,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    # Only use the labels that appear in the data
    classes = classes[unique_labels(y_true, y_pred)]
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    fig, ax = plt.subplots()
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax

In [ ]:
#Look at confusion matrix of results
y_hat_train = model_three.predict(X_nn_train)
y_hat_test = model_three.predict(X_nn_test)

classes = [0, 1]

cm = confusion_matrix(labels_train, np.round_(y_hat_train, decimals=0))
plot_conf_matrix(cm, classes )
plt.savefig('Confusion Matrix')
plt.show();